# Modelación y Simulación - Laboratorio 6
## Simulación de Eventos Discretos: Servidores

---
### Integrantes:
- Gustavo Andrés González 21438
- Diego Alberto Leiva 21752
- José Pablo Orellana 21970
- - -

## Librerias

In [2]:
import simpy
import random
import numpy as np

## Parametros y configuraciones

In [3]:
# Parámetros generales
LAMBDA = 2400 / 60  # Solicitudes por segundo
SIMULATION_TIME = 3600  # Tiempo de simulación en segundos (1 hora)
SEED = 42  # Semilla para reproducibilidad

# Configuración de los servidores
SERVIDOR_MMC = 1  # Mountain Mega Computing tiene un solo servidor
CAPACIDAD_MMC = 100  # Capacidad de Mountain Mega (solicitudes por segundo)

SERVIDORES_PIZZITA = 10  # 10 servidores para Pizzita computing
CAPACIDAD_PIZZITA = CAPACIDAD_MMC / 10  # Cada servidor de Pizzita tiene 1/10 de la capacidad de MMC

# Variables globales para la recolección de datos
class Metrics:
    def __init__(self):
        self.num_requests = 0
        self.total_waiting_time = 0
        self.queue_length = 0
        self.busy_time = 0
        self.idle_time = 0
        self.last_exit_time = 0

## Simulaciones

In [4]:
def handle_request(env, server, capacity, metrics):
    """
    Manejo de una solicitud en el servidor.
    """
    arrival_time = env.now
    metrics.num_requests += 1
    
    with server.request() as request:
        # Cola
        yield request
        waiting_time = env.now - arrival_time
        metrics.total_waiting_time += waiting_time
        
        # Tiempo de servicio
        service_time = random.expovariate(1 / capacity)
        yield env.timeout(service_time)
        
        metrics.busy_time += service_time
        metrics.last_exit_time = env.now

In [5]:
def arrival_process(env, server, capacity, metrics):
    """
    Proceso de llegada de solicitudes al sistema.

    Args:
        env: entorno de simulación
        server: servidor al que se enviarán las solicitudes
        capacity: capacidad del servidor
        metrics: objeto para recolectar métricas

    Yields:
        eventos de llegada de solicitudes
    """
    while True:
        # Generar nueva solicitud
        inter_arrival_time = random.expovariate(LAMBDA)
        yield env.timeout(inter_arrival_time)
        
        env.process(handle_request(env, server, capacity, metrics))

In [6]:

def run_simulation(num_servers, capacity, simulation_time=SIMULATION_TIME):
    """
    Función para correr la simulación de un sistema de colas.

    Args:
        num_servers: cantidad de servidores en el sistema
        capacity: capacidad de cada servidor
        simulation_time: tiempo de simulación

    Returns:
        diccionario con las métricas de la simulación
    """
    random.seed(SEED)
    env = simpy.Environment()
    
    server = simpy.Resource(env, capacity=num_servers)
    metrics = Metrics()
    
    env.process(arrival_process(env, server, capacity, metrics))
    
    env.run(until=simulation_time)
    
    metrics.idle_time = simulation_time * num_servers - metrics.busy_time
    avg_waiting_time = metrics.total_waiting_time / metrics.num_requests if metrics.num_requests > 0 else 0
    avg_queue_length = metrics.total_waiting_time / simulation_time
    
    return {
        "Solicitudes atendidas": metrics.num_requests,
        "Tiempo ocupado": metrics.busy_time,
        "Tiempo desocupado": metrics.idle_time,
        "Tiempo total en cola": metrics.total_waiting_time,
        "Tiempo promedio en cola": avg_waiting_time,
        "Promedio solicitudes en cola": avg_queue_length,
        "Última solicitud salida": metrics.last_exit_time
    }


## Resultados

In [7]:
def pretty_print(metrics):
    """
    Función para imprimir las métricas de la simulación de forma legible.

    Args:
        metrics: diccionario con las métricas de la simulación

    Returns:
        None
    """
    print("Métricas de la simulación:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")

In [8]:
# Simulación para Mountain Mega Computing
result_mmc = run_simulation(SERVIDOR_MMC, CAPACIDAD_MMC)
print("Mountain Mega Computing:")
pretty_print(result_mmc)

Mountain Mega Computing:
Métricas de la simulación:
Solicitudes atendidas: 143916.00
Tiempo ocupado: 3544.34
Tiempo desocupado: 55.66
Tiempo total en cola: 78898.24
Tiempo promedio en cola: 0.55
Promedio solicitudes en cola: 21.92
Última solicitud salida: 3544.37


In [9]:
# Simulación para Pizzita Computing
result_pizzita = run_simulation(SERVIDORES_PIZZITA, CAPACIDAD_PIZZITA)
print("\nPizzita Computing:")
pretty_print(result_pizzita)


Pizzita Computing:
Métricas de la simulación:
Solicitudes atendidas: 143949.00
Tiempo ocupado: 35923.21
Tiempo desocupado: 76.79
Tiempo total en cola: 6381590.65
Tiempo promedio en cola: 44.33
Promedio solicitudes en cola: 1772.66
Última solicitud salida: 3599.70


## Tasks
1. Modele, simule y analice el comportamiento de ambos sistemas durante una hora de ejecución de C3, y
para cada sistema responda: 

    a. ¿Cuántas solicitudes atendió cada servidor?  
        Mountain Mega Computing: 143,916 solicitudes.  
        Pizzita Computing: 143,949 solicitudes.  

    b. ¿Cuánto tiempo estuvo cada servidor ocupado?  
        Mountain Mega Computing: 3,544.34 segundos.  
        Pizzita Computing: 35,923.21 segundos.  

    c. ¿Cuánto tiempo estuvo cada servidor desocupado (iddle)?  
        Mountain Mega Computing: 55.66 segundos.  
        Pizzita Computing: 76.79 segundos.  

    d. ¿Cuánto tiempo en total estuvieron las solicitudes en cola?  
        Mountain Mega Computing: 78,898.24 segundos.  
        Pizzita Computing: 6,381,590.65 segundos.  

    e. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?  
        Mountain Mega Computing: 0.55 segundos.  
        Pizzita Computing: 44.33 segundos.  

    f. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?  
        Mountain Mega Computing: 21.92 solicitudes.  
        Pizzita Computing: 1,772.66 solicitudes.  

    g. ¿Cuál es el momento de la salida de la última solicitud?  
        Mountain Mega Computing: 3,544.37 segundos.  
        Pizzita Computing: 3,599.70 segundos.  


2. Determine empíricamente cuántos servidores se necesitaría “alquilar” en Pizzita computing para asegurar
que siempre habrá al menos un servidor disponible para atender una solicitud dada (en otras palabras, una
solicitud nunca tiene que esperar en cola)

    Actualmente, el sistema tiene un promedio de 1,772.66 solicitudes en cola cada segundo, lo que significa que el servidor no puede atender todas las solicitudes a tiempo.

    Para resolver este problema, calculamos las solicitudes atendidas por segundo y el número de servidores necesarios.

    Promedio de solicitudes por segundo = 143,949/3599.7 = 40.  
    Tiempo promedio en cola = 44.33 segundos.  

    Este tiempo de espera indica que hay un embotellamiento, por lo que necesitaríamos más servidores. Para determinar el número de servidores requeridos, asumimos que el número de solicitudes que puede manejar un servidor sin generar espera es el inverso del tiempo de servicio promedio.

    Número de servidores necesarios  N :

    $$
    N = \frac{1772.66}{40} = 44.32
    $$

3. Se espera que a partir del tercer año del lanzamiento de su aplicación, la cantidad de usuarios sufra un alza,
y por tanto deberán atender como máximo 6000 solicitudes por minuto. Resuelva el inciso 1 y 2 para esta
nueva configuración.

In [10]:
# Parámetros generales
LAMBDA = 6000 / 60  # Solicitudes por segundo
SIMULATION_TIME = 3600  # Tiempo de simulación en segundos (1 hora)
SEED = 42  # Semilla para reproducibilidad

# Configuración de los servidores
SERVIDOR_MMC = 1  # Mountain Mega Computing tiene un solo servidor
CAPACIDAD_MMC = 100  # Capacidad de Mountain Mega (solicitudes por segundo)

SERVIDORES_PIZZITA = 10  # 10 servidores para Pizzita computing
CAPACIDAD_PIZZITA = CAPACIDAD_MMC / 10  # Cada servidor de Pizzita tiene 1/10 de la capacidad de MMC

In [11]:
# Simulación para Mountain Mega Computing
result_mmc = run_simulation(SERVIDOR_MMC, CAPACIDAD_MMC)
print("Mountain Mega Computing:")
pretty_print(result_mmc)

Mountain Mega Computing:
Métricas de la simulación:
Solicitudes atendidas: 359587.00
Tiempo ocupado: 3541.95
Tiempo desocupado: 58.05
Tiempo total en cola: 53152.67
Tiempo promedio en cola: 0.15
Promedio solicitudes en cola: 14.76
Última solicitud salida: 3541.96


In [12]:
# Simulación para Pizzita Computing
result_pizzita = run_simulation(SERVIDORES_PIZZITA, CAPACIDAD_PIZZITA)
print("\nPizzita Computing:")
pretty_print(result_pizzita)


Pizzita Computing:
Métricas de la simulación:
Solicitudes atendidas: 359641.00
Tiempo ocupado: 35897.04
Tiempo desocupado: 102.96
Tiempo total en cola: 6255922.31
Tiempo promedio en cola: 17.39
Promedio solicitudes en cola: 1737.76
Última solicitud salida: 3595.99


Promedio de solicitudes por segundo = 359641.00/3595.99 = 100
Tiempo promedio en cola = 17.39 segundos.  
N = 1737.76/100 = 18

4. Emita una recomendación para la junta directiva

    Mountain Mega Computing es un proveedor mucho más eficiente en cuanto a tiempo de respuesta y manejo de solicitudes, mostrando un menor tiempo en cola y mayor eficiencia operativa con menos servidores.  

    Pizzita Computing tiene tiempos de espera significativamente mayores y necesitaría aumentar dramáticamente el número de servidores para poder competir con Mountain Mega Computing en términos de rendimiento y evitar tiempos de espera elevados.